In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.1`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Functions")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
23/08/20 17:13:34 WARN Utils: Your hostname, Air.local resolves to a loopback address: 127.0.0.1; using 192.168.88.21 instead (on interface en5)
23/08/20 17:13:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/08/20 17:13:34 INFO SparkContext: Running Spark version 3.4.1
23/08/20 17:13:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/20 17:13:34 INFO ResourceUtils: ==============================================================
23/08/20 17:13:34 INFO ResourceUtils: No custom resources configured for spark.driver.
23/08/20 17:13:34 INFO ResourceUtils: ==============================================================
23/08/20 17:13:34 INFO SparkContext: Submitted application: Functions
23/08/20 17:13:34 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, 

spark: SparkSession = org.apache.spark.sql.SparkSession@1c4c06f8
import spark.implicits._

In [4]:
val data = Seq(
    ("Jean-Georges", "Perrin", "NC", 1, 300, 1551903533),
    ("Jean-Georges", "Perrin", "NC", 2, 120, 1551903567),
    ("Jean-Georges", "Perrin", "CA" ,4, 75, 1551903599),
    ("Holden", "Karau", "CA" , 6, 37, 1551904299),
    ("Ginni", "Rometty", "NY", 7, 91, 1551916792),
    ("Holden", "Karau", "CA", 4, 153, 1552876129)
).toDF("firstName", "lastName", "state", "quantity", "revenue", "timestamp")

data.show()

23/08/20 17:13:36 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
23/08/20 17:13:36 INFO SharedState: Warehouse path is 'file:/Users/vadim/Projects/OtusTeam/SparkDeveloper/lesson-07/Notebooks/spark-warehouse'.
23/08/20 17:13:37 INFO CodeGenerator: Code generated in 183.058166 ms
23/08/20 17:13:38 INFO CodeGenerator: Code generated in 7.691208 ms
23/08/20 17:13:38 INFO CodeGenerator: Code generated in 10.3405 ms


+------------+--------+-----+--------+-------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|
+------------+--------+-----+--------+-------+----------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|
|      Holden|   Karau|   CA|       6|     37|1551904299|
|       Ginni| Rometty|   NY|       7|     91|1551916792|
|      Holden|   Karau|   CA|       4|    153|1552876129|
+------------+--------+-----+--------+-------+----------+



data: DataFrame = [firstName: string, lastName: string ... 4 more fields]

In [5]:
data.printSchema

root
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- state: string (nullable = true)
 |-- quantity: integer (nullable = false)
 |-- revenue: integer (nullable = false)
 |-- timestamp: integer (nullable = false)



## Array Functions

In [6]:
val datak2 = data
                .withColumn("k2v", array($"firstName", $"lastName", $"state"))
                .withColumn("k2n", array(lit("FirstName"), lit("LastName"), lit("State")))

datak2.show(false)

23/08/20 17:13:39 INFO CodeGenerator: Code generated in 11.174791 ms
23/08/20 17:13:39 INFO CodeGenerator: Code generated in 13.683333 ms


+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2v                       |k2n                         |
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[Jean-Georges, Perrin, CA]|[FirstName, LastName, State]|
|Holden      |Karau   |CA   |6       |37     |1551904299|[Holden, Karau, CA]       |[FirstName, LastName, State]|
|Ginni       |Rometty |NY   |7       |91     |1551916792|[Ginni, Rometty, NY]      |[FirstName, LastName, State]|
|Holden      |Karau   |CA   |4       |153    |1552876129|[Holden, Karau, CA]       |[Fir

datak2: DataFrame = [firstName: string, lastName: string ... 6 more fields]

In [7]:
datak2.where(array_contains($"k2v", "CA")).show(false)

23/08/20 17:13:39 INFO CodeGenerator: Code generated in 8.04425 ms


+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2v                       |k2n                         |
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[Jean-Georges, Perrin, CA]|[FirstName, LastName, State]|
|Holden      |Karau   |CA   |6       |37     |1551904299|[Holden, Karau, CA]       |[FirstName, LastName, State]|
|Holden      |Karau   |CA   |4       |153    |1552876129|[Holden, Karau, CA]       |[FirstName, LastName, State]|
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+



## Map Functions

In [8]:
val datak2m = data.withColumn("k2m", map(lit("FirstName"), $"firstName", lit("LastName"), $"lastName", lit("State"), $"state"))

datak2m.show(false)

23/08/20 17:13:39 INFO CodeGenerator: Code generated in 9.522583 ms
23/08/20 17:13:39 INFO CodeGenerator: Code generated in 15.85875 ms


+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2m                                                         |
+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|{FirstName -> Jean-Georges, LastName -> Perrin, State -> CA}|
|Holden      |Karau   |CA   |6       |37     |1551904299|{FirstName -> Holden, LastName -> Karau, State -> CA}       |
|Ginni       |Rometty |NY   |7       |91     |1551916792|{FirstName -> Ginni, LastName -> Rometty, State -> NY}      |
|Holden      |Karau   |CA   |4       |153    |15

datak2m: DataFrame = [firstName: string, lastName: string ... 5 more fields]

In [9]:
val datak2ma = datak2.withColumn("k2m", map_from_arrays($"k2n", $"k2v"))

datak2ma.show(false)

23/08/20 17:13:40 INFO CodeGenerator: Code generated in 8.999666 ms
23/08/20 17:13:40 INFO CodeGenerator: Code generated in 19.821375 ms


+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+------------------------------------------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2v                       |k2n                         |k2m                                                         |
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+------------------------------------------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[Jean-Georges, Perrin, CA]|[FirstName, LastName, State]|{FirstName -

datak2ma: DataFrame = [firstName: string, lastName: string ... 7 more fields]

## Date and Timestamp Functions

In [10]:
val datat = data
                .withColumn("recordTimestamp", to_timestamp($"timestamp"))
                .withColumn("current", current_timestamp)

datat.show(false)

+------------+--------+-----+--------+-------+----------+-------------------+-----------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |recordTimestamp    |current                |
+------------+--------+-----+--------+-------+----------+-------------------+-----------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|2019-03-06 23:18:53|2023-08-20 17:13:40.654|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|2019-03-06 23:19:27|2023-08-20 17:13:40.654|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|2019-03-06 23:19:59|2023-08-20 17:13:40.654|
|Holden      |Karau   |CA   |6       |37     |1551904299|2019-03-06 23:31:39|2023-08-20 17:13:40.654|
|Ginni       |Rometty |NY   |7       |91     |1551916792|2019-03-07 02:59:52|2023-08-20 17:13:40.654|
|Holden      |Karau   |CA   |4       |153    |1552876129|2019-03-18 05:28:49|2023-08-20 17:13:40.654|
+------------+--------+-----+--------+-------+----------+-------------------+-----

datat: DataFrame = [firstName: string, lastName: string ... 6 more fields]

In [11]:
datat.select($"recordTimestamp", $"current", datediff($"current", $"recordTimestamp")).show(false)

23/08/20 17:13:41 INFO CodeGenerator: Code generated in 4.135208 ms
23/08/20 17:13:41 INFO CodeGenerator: Code generated in 4.452167 ms


+-------------------+-------------------+----------------------------------+
|recordTimestamp    |current            |datediff(current, recordTimestamp)|
+-------------------+-------------------+----------------------------------+
|2019-03-06 23:18:53|2023-08-20 17:13:41|1628                              |
|2019-03-06 23:19:27|2023-08-20 17:13:41|1628                              |
|2019-03-06 23:19:59|2023-08-20 17:13:41|1628                              |
|2019-03-06 23:31:39|2023-08-20 17:13:41|1628                              |
|2019-03-07 02:59:52|2023-08-20 17:13:41|1627                              |
|2019-03-18 05:28:49|2023-08-20 17:13:41|1616                              |
+-------------------+-------------------+----------------------------------+



## JSON Functions

In [12]:
val datak2j = datak2m.withColumn("k2j", to_json($"k2m"))

datak2j.show(false)

23/08/20 17:13:41 INFO CodeGenerator: Code generated in 7.491334 ms
23/08/20 17:13:41 INFO CodeGenerator: Code generated in 11.462542 ms


+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+-------------------------------------------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2m                                                         |k2j                                                          |
+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+-------------------------------------------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|{"FirstName":"Jean-Georges","LastName":"Perrin","State":"NC"}|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|{"FirstName":"Jean-Georges","LastName":"Perrin","State":"NC"}|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|{FirstName -> Jean-Georges, LastName -

datak2j: DataFrame = [firstName: string, lastName: string ... 6 more fields]

## Generator Functions

In [13]:
val data2 = data
                .withColumn("a1", array(lit(1), lit(2), lit(3), lit(4), lit(5)))
                .withColumn("a2", lit((1 to 5).toArray))

data2.show(false)

+------------+--------+-----+--------+-------+----------+---------------+---------------+
|firstName   |lastName|state|quantity|revenue|timestamp |a1             |a2             |
+------------+--------+-----+--------+-------+----------+---------------+---------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Holden      |Karau   |CA   |6       |37     |1551904299|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Ginni       |Rometty |NY   |7       |91     |1551916792|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Holden      |Karau   |CA   |4       |153    |1552876129|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
+------------+--------+-----+--------+-------+----------+---------------+---------------+



data2: DataFrame = [firstName: string, lastName: string ... 6 more fields]

In [14]:
data2.withColumn("dummy", explode($"a1")).show(false)

23/08/20 17:13:41 INFO CodeGenerator: Code generated in 4.508667 ms
23/08/20 17:13:42 INFO CodeGenerator: Code generated in 16.747041 ms
23/08/20 17:13:42 INFO CodeGenerator: Code generated in 12.167208 ms
23/08/20 17:13:42 INFO SparkContext: Starting job: show at cell14.sc:1
23/08/20 17:13:42 INFO DAGScheduler: Got job 0 (show at cell14.sc:1) with 1 output partitions
23/08/20 17:13:42 INFO DAGScheduler: Final stage: ResultStage 0 (show at cell14.sc:1)
23/08/20 17:13:42 INFO DAGScheduler: Parents of final stage: List()
23/08/20 17:13:42 INFO DAGScheduler: Missing parents: List()
23/08/20 17:13:42 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[2] at show at cell14.sc:1), which has no missing parents
23/08/20 17:13:42 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 12.9 KiB, free 2004.6 MiB)
23/08/20 17:13:42 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 5.2 KiB, free 2004.6 MiB)
23/08/20 17:13:42 INFO B

+------------+--------+-----+--------+-------+----------+---------------+---------------+-----+
|firstName   |lastName|state|quantity|revenue|timestamp |a1             |a2             |dummy|
+------------+--------+-----+--------+-------+----------+---------------+---------------+-----+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|1    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|2    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|3    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|4    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|5    |
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|1    |
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|2    |
|Jean-Georges|Perrin  |NC   |2       |12

## Window Functions

In [15]:
import org.apache.spark.sql.expressions.Window

val windowSpec  = Window.partitionBy("firstName", "lastName").orderBy("state")

import org.apache.spark.sql.expressions.Window
windowSpec: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@19fd7759

In [16]:
val dataw = data.withColumn("row_number", row_number.over(windowSpec))

dataw.show

23/08/20 17:13:43 INFO CodeGenerator: Code generated in 5.350041 ms
23/08/20 17:13:43 INFO DAGScheduler: Registering RDD 5 (show at cell16.sc:3) as input to shuffle 0
23/08/20 17:13:43 INFO DAGScheduler: Got map stage job 2 (show at cell16.sc:3) with 6 output partitions
23/08/20 17:13:43 INFO DAGScheduler: Final stage: ShuffleMapStage 2 (show at cell16.sc:3)
23/08/20 17:13:43 INFO DAGScheduler: Parents of final stage: List()
23/08/20 17:13:43 INFO DAGScheduler: Missing parents: List()
23/08/20 17:13:43 INFO DAGScheduler: Submitting ShuffleMapStage 2 (MapPartitionsRDD[5] at show at cell16.sc:3), which has no missing parents
23/08/20 17:13:43 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 8.9 KiB, free 2004.6 MiB)
23/08/20 17:13:43 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 4.7 KiB, free 2004.6 MiB)
23/08/20 17:13:43 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on 192.168.88.21:64317 (size: 4.7 KiB, fr

+------------+--------+-----+--------+-------+----------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_number|
+------------+--------+-----+--------+-------+----------+----------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|         1|
|      Holden|   Karau|   CA|       6|     37|1551904299|         1|
|      Holden|   Karau|   CA|       4|    153|1552876129|         2|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|         1|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|         2|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|         3|
+------------+--------+-----+--------+-------+----------+----------+



dataw: DataFrame = [firstName: string, lastName: string ... 5 more fields]

In [17]:
dataw.where($"row_number" === 1).show

23/08/20 17:13:44 INFO DAGScheduler: Registering RDD 13 (show at cell17.sc:1) as input to shuffle 1
23/08/20 17:13:44 INFO DAGScheduler: Got map stage job 4 (show at cell17.sc:1) with 6 output partitions
23/08/20 17:13:44 INFO DAGScheduler: Final stage: ShuffleMapStage 5 (show at cell17.sc:1)
23/08/20 17:13:44 INFO DAGScheduler: Parents of final stage: List()
23/08/20 17:13:44 INFO DAGScheduler: Missing parents: List()
23/08/20 17:13:44 INFO DAGScheduler: Submitting ShuffleMapStage 5 (MapPartitionsRDD[13] at show at cell17.sc:1), which has no missing parents
23/08/20 17:13:44 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 8.9 KiB, free 2004.5 MiB)
23/08/20 17:13:44 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 4.7 KiB, free 2004.5 MiB)
23/08/20 17:13:44 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on 192.168.88.21:64317 (size: 4.7 KiB, free: 2004.6 MiB)
23/08/20 17:13:44 INFO SparkContext: Created broad

+------------+--------+-----+--------+-------+----------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_number|
+------------+--------+-----+--------+-------+----------+----------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|         1|
|      Holden|   Karau|   CA|       6|     37|1551904299|         1|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|         1|
+------------+--------+-----+--------+-------+----------+----------+



In [18]:
dataw.withColumn("revenue2", nth_value($"revenue", 2).over(windowSpec)).show

23/08/20 17:13:44 INFO DAGScheduler: Registering RDD 21 (show at cell18.sc:1) as input to shuffle 2
23/08/20 17:13:44 INFO DAGScheduler: Got map stage job 6 (show at cell18.sc:1) with 6 output partitions
23/08/20 17:13:44 INFO DAGScheduler: Final stage: ShuffleMapStage 8 (show at cell18.sc:1)
23/08/20 17:13:44 INFO DAGScheduler: Parents of final stage: List()
23/08/20 17:13:44 INFO DAGScheduler: Missing parents: List()
23/08/20 17:13:44 INFO DAGScheduler: Submitting ShuffleMapStage 8 (MapPartitionsRDD[21] at show at cell18.sc:1), which has no missing parents
23/08/20 17:13:44 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 8.9 KiB, free 2004.5 MiB)
23/08/20 17:13:44 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 4.7 KiB, free 2004.5 MiB)
23/08/20 17:13:44 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on 192.168.88.21:64317 (size: 4.7 KiB, free: 2004.6 MiB)
23/08/20 17:13:44 INFO SparkContext: Created broad

+------------+--------+-----+--------+-------+----------+----------+--------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_number|revenue2|
+------------+--------+-----+--------+-------+----------+----------+--------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|         1|    null|
|      Holden|   Karau|   CA|       6|     37|1551904299|         1|     153|
|      Holden|   Karau|   CA|       4|    153|1552876129|         2|     153|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|         1|    null|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|         2|     300|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|         3|     300|
+------------+--------+-----+--------+-------+----------+----------+--------+

